<h2>Emotion Recognition Model</h2>
<h4>by A Bhagyalaxmi</h4>
Using <i>Convolutional Neural Network</i> creating an Emotion Detection Web Application which can classify the emotion of a person into 7 different categories: Happy, Sad, Neutral, Angry, Fear, Surprise and Disgust.

Dataset: FER2013

Tech stack: TensorFlow, OpenCV, numpy, 

<h3>Prerequisties</h3>

In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
import tensorflow as tf
from livelossplot import PlotLossesKeras
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.8.0


<h3>Normalising the dataset</h3>

In [9]:
train_dir = 'train'
val_dir = 'test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 25725 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


<h3>Using Convolutional Neural Network model to train the dataset to classifying the emotions.</h3>

In [10]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1))) #Input image of dimension 48×48
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))

emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))

emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())

emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))

emotion_model.add(Dense(7, activation='softmax')) # Output layer containing of 7 different emotions 

<h3>Compile the model using accuracy to measure model performance.</h3>

In [11]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=22400 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
350/350 [==============================] - 229s 653ms/step - loss: 1.6994 - accuracy: 0.2732 - val_loss: 1.9172 - val_accuracy: 0.2497
Epoch 2/50
350/350 [==============================] - 93s 266ms/step - loss: 1.6259 - accuracy: 0.3255 - val_loss: 1.8873 - val_accuracy: 0.3468
Epoch 3/50
350/350 [==============================] - 90s 258ms/step - loss: 1.5274 - accuracy: 0.3938 - val_loss: 1.8493 - val_accuracy: 0.3838
Epoch 4/50
350/350 [==============================] - 91s 261ms/step - loss: 1.4536 - accuracy: 0.4306 - val_loss: 1.7300 - val_accuracy: 0.4032
Epoch 5/50
350/350 [==============================] - 91s 259ms/step - loss: 1.4039 - accuracy: 0.4518 - val_loss: 1.6932 - val_accuracy: 0.4223
Epoch 6/50
350/350 [==============================] - 90s 258ms/step - loss: 1.3622 - accuracy: 0.4686 - val_loss: 1.6320 - val_accuracy: 0.4319
Epoch 7/50
350/350 [==============================] - 90s 257ms/step - loss: 1.3127 - accuracy: 0.4860 - val_loss: 1.6295 - val_a

In [ ]:
# Saving the model as h5 file.
emotion_model.save('model.h5')

In [ ]:
# JSON file for transmitting data between the web application and the model
model_json = emotion_model.to_json()
with open("model_try.json","w") as json_file:
    json_file.write(model_json)
    